# Regresión Logística para el Titanic

En este notebook buscamos construir un modelo de regresión logística para predecir la variable survival que nos indica si el pasajero ha sobrevivido (1) al naufragio del Titanic o ha fallecido (0).

En este notebook usaremos los siguientes módulos:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 

Comenzamos cargando los datos y preprocesando las variables.

## Preprocesamiento

### Valores perdidos

Empezamos observando el número de valores perdidos para cada variable:

In [ ]:
titanic_data = pd.read_csv('./data/titanic.csv')

In [ ]:
titanic_data.head()

In [ ]:
titanic_data.shape

In [ ]:
titanic_data.isnull().sum()

En este caso completaremos las variables perdidas edad y puerto de embarcamiento pues pueden resultar interesantes.  Para la variable edad empleamos la mediana para sustituir:

In [ ]:
titanic_data['Age'].median() # calculamos la mediana

In [ ]:
titanic_data['Age'].fillna(titanic_data['Age'].median(), inplace=True) # rellenamos los valores perdidos con la mediana

La variable del puerto de embarque la completamos con el valor más habitual, es decir, con la moda:

In [ ]:
titanic_data['Embarked'].mode()[0] # calculamos la moda

In [ ]:
titanic_data['Embarked'].fillna(titanic_data['Embarked'].mode()[0], inplace=True)

Podemos estudiar la variable cabina:

In [ ]:
titanic_data['Cabin'].value_counts().sort_index()

Observamos que la variable cabina no parece contener mucha información relevante. Toma 147 valores distintos para menos de 900 variables por lo que la descartaremos.

### Descarte de variables

Como mencioamos antes podemos prescindir de la variable cabina. Además las variables PassengerId, Name, Ticket y Fare tampoco parecen aportar mucha información por lo que nos quedamos con las variables restantes:

In [ ]:
titanic_data.head()

In [ ]:
cleaned_data = titanic_data[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Embarked']]

Generamos las variables dummies que usaremos para codificar las tres variables categóricas:

In [ ]:
final_data= pd.get_dummies(cleaned_data, columns=['Pclass', 'Sex', 'Embarked' ], drop_first= True)  #automatiza la generación de dummies

In [ ]:
final_data.head()

Este será el dataset con el que trabajaremos. Procedemos a continuación a dividir en datos de entrenamiento y datos de validación.

## Dividiendo en datos de entrenamiento y datos de validación 

Con el fin de poder evaluar la calidad de nuestro modelo dividimos en datos de entrenamiento y datos de validación:

In [ ]:
X = final_data.drop('Survived', axis=1)

In [ ]:
y = final_data['Survived']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

__Nota.__ La semilla random_starte nos permitirá replicar esta partición en futuros notebooks.

## Preprocesamiento final

Realizamos finalmente el escalado de variables que no se puede realizar hasta que se proceda a la división entre el dataset de entrenamiento y el de validación para no generar sesgo.

### Escalado de variables

Procedemos a continuación a escalar las variables numéricas para intentar que el método resulte algo más estable:

In [ ]:
from sklearn.preprocessing import StandardScaler
ss= StandardScaler()

In [ ]:
import warnings
warnings.filterwarnings('ignore')
features= ['Age', 'SibSp', 'Parch'] #variables requieren ser escaladas

X_train[features]= ss.fit_transform(X_train[features]) #transformación de entrenamiento
X_test[features]= ss.fit_transform(X_test[features]) #transformación de validación

Tras esto ya tenemos nuestros datos listos para la construcción del modelo:

In [ ]:
X_train.head()

Guardamo los datasets de entrenamiento y validación para poder reutilizarlos en futuros notebooks:

In [ ]:
X_train.to_csv('./data/xtrain_tit.csv', index=False)
X_test.to_csv('./data/xtest_tit.csv', index=False)
y_train.to_csv('./data/ytrain_tit.csv', index=False)
y_test.to_csv('./data/ytest_tit.csv', index=False)

## Construcción del modelo

Ahora que ya tenemos nuestro dataset preparado tras tratar los valores perdidos, escalarlo, generar las variables dummy etc ha llegado el momento de ajustar nuestro modelo de regresión logística. Este será el primer modelo de clasificación que ajustemos:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

Como es habitual comenzamos instanciando el modelo:

In [ ]:
logreg = LogisticRegression()

Tras ello lo ajustamos sobre nuestros datos de entrenamiento pasándole las variables predictoras y la variable objetivo y:

In [ ]:
logreg.fit(X_train, y_train)

Podemos observar los coeficientes que se han asociado a este modelo mediante el entrenamiento:

In [ ]:
print(logreg.coef_)

Hasta aquí ya tenemos un modelo capaz de realizar predicciones. A continuación predeciremos sobre el conjunto de validación para evaluar la calidad de nuestro modelo:

In [ ]:
y_pred = logreg.predict(X_test)

In [ ]:
X_test.head()

In [ ]:
y_pred

## Evaluando la calidad del modelo

Como estamos trabajando con dato etiquetado podemos comparar el resultado real **y_test** con el resultado predicho **y_pred** para poder evaluar la calidad del modelo:

In [ ]:
logreg.score(X_test, y_test)

Observamos que la precisión a simple vista es bastante buena, acierta en un 83% de los casos. Como nos encontramos ante datos desbalanceados podemos proceder a calcular otras métricas para constatar si este primer buen resultado se consolida. Construímos la matriz de confusión:

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

Observamos que los resultados son muy buenos. Nuestra regresión acierta de una manera bastante equilibrada. Podemos calcular los coeficientes observados en la parte teórica para poder posteriormente comparar de manera objetiva y sencilla otros modelos que estudiaremos en el futuro. Sklearn nos provee de un método para calcularlos y organizarlos de manera automática:

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

Por último podemos calcular la curva ROC que nos será también de gran utilidad para evaluar la calidad del modelo:

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test)) # el cálculo del área bajo la curva está automatizado
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1]) #la curva ROC se genera en esta línea
plt.figure() # a partir de aquí se da formato a la gráfica
plt.plot(fpr, tpr, label='Regresión Logística (Área bajo la curva = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Ratio de falsos positivos')
plt.ylabel('Ratio de verdaderos positivos')
plt.title('Curva ROC')
plt.legend(loc="lower right")
#plt.savefig('Log_ROC') si descomentas esta línea puedes guardar la gráfica
plt.show()

Observamos que el área bajo la curva aparece calculado y es de 0,83 un muy buen resultado que intentaremos mejorar en el futuro.